In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

# Simulated data: moves made by players
players_behavior = [
    ["e2e4", "e7e5", "d1h5", "g8f6"],  # Player 1
    ["d2d4", "f1c4", "h5e5", "d7d6"],  # Player 2
    ["c2c4", "f2f4", "e2e4", "g1f3"],  # Player 3
    ["e4e5", "d2d4", "h2h4", "f7f5"],  # Player 4
    ["b2b4", "f2f3", "g1h3", "a2a3"],  # Player 5
]

# Vectorize move sequences for clustering
vectorizer = CountVectorizer(analyzer='word', tokenizer=lambda x: x, preprocessor=lambda x: x)
sequence_features = vectorizer.fit_transform(players_behavior).toarray()

# Cluster players based on their move patterns
n_clusters = 2  # Assume we want 2 clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
player_clusters = kmeans.fit_predict(sequence_features)

# Print cluster assignments
for i, cluster_id in enumerate(player_clusters):
    print(f"Player {i + 1} is in Cluster {cluster_id}")


Player 1 is in Cluster 0
Player 2 is in Cluster 0
Player 3 is in Cluster 0
Player 4 is in Cluster 0
Player 5 is in Cluster 1


c:\Users\yassi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# Encode moves into integers
tokenizer = LabelEncoder()
all_moves = [move for seq in players_behavior for move in seq]
tokenizer.fit(all_moves)

# Function to prepare data for each cluster
def prepare_lstm_data(cluster_id):
    cluster_sequences = [
        seq for i, seq in enumerate(players_behavior) if player_clusters[i] == cluster_id
    ]
    tokenized_sequences = [
        [tokenizer.transform([move])[0] for move in sequence] for sequence in cluster_sequences
    ]
    X, y = [], []
    for seq in tokenized_sequences:
        for i in range(len(seq) - 1):  # Create input-output pairs
            X.append(seq[:i + 1])
            y.append(seq[i + 1])
    X = pad_sequences(X, maxlen=10)  # Pad sequences to a max length
    y = to_categorical(y, num_classes=len(all_moves))  # One-hot encode outputs
    return np.array(X), np.array(y)

# Train LSTM for each cluster
lstm_models = {}
for cluster_id in range(n_clusters):
    X_train, y_train = prepare_lstm_data(cluster_id)
    model = Sequential([
        Embedding(input_dim=len(all_moves), output_dim=32, input_length=10),
        LSTM(64, return_sequences=False),
        Dense(len(all_moves), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
    lstm_models[cluster_id] = model

print("LSTM models trained for all clusters.")


c:\Users\yassi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.0000e+00 - loss: 2.9964
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.0000e+00 - loss: 2.9911
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.2500 - loss: 2.9858
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.1667 - loss: 2.9803
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.1667 - loss: 2.9746
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.1667 - loss: 2.9686
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.0833 - loss: 2.9621
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.0833 - loss: 2.9551
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.0833 - loss: 2.9473
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.0833 - loss: 2.9386
Epoch 1/10


c:\Users\yassi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3333 - loss: 2.9815
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3333 - loss: 2.9649
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3333 - loss: 2.9476
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3333 - loss: 2.9292
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3333 - loss: 2.9092
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3333 - loss: 2.8870
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3333 - loss: 2.8621
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3333 - loss: 2.8337
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3333 - loss: 2.8009
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3333 - loss: 2.7627
LSTM models trained for all clusters.


In [6]:
# New player's move sequence
new_player_moves = ["e2e4", "e7e5", "d1h5"]

# Map to cluster
new_player_features = vectorizer.transform([new_player_moves]).toarray()
predicted_cluster = kmeans.predict(new_player_features)[0]
print(f"New player is in Cluster {predicted_cluster}")

# Predict next moves using the corresponding cluster's LSTM model
tokenized_input = tokenizer.transform(new_player_moves)
padded_input = pad_sequences([tokenized_input], maxlen=10)
predicted_probs = lstm_models[predicted_cluster].predict(padded_input)
predicted_moves = [tokenizer.inverse_transform([i])[0] for i in np.argsort(predicted_probs[0])[-3:]]

print(f"Predicted next moves for the new player: {predicted_moves}")


New player is in Cluster 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step
Predicted next moves for the new player: ['h2h4', 'd2d4', 'g1f3']
